In [1]:
import numpy as np
import torch
from torch import nn, optim, autograd
from torch.nn import functional as F
from pyDOE import lhs
import scipy.io
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.gridspec as gridspec
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable

from models_all import *

#Paper reproduction
torch.manual_seed(1234)
torch.cuda.manual_seed(1234)
np.random.seed(1234)

# Data For PINNs

In [2]:
N_train = 5000
N_bound = 100

np.random.seed(123)
train_x_y_t = lhs(3,N_train)
np.random.seed(12345)
bound_x_y = lhs(2,N_bound)
bound_x_y_t = np.concatenate((bound_x_y,np.zeros((N_bound,1))),1)

In [3]:
train_x_y_t = torch.from_numpy(train_x_y_t).float()
bound_x_y = torch.from_numpy(bound_x_y).float()
bound_x_y_t = torch.from_numpy(bound_x_y_t).float()

x = train_x_y_t[:,0:1]
y = train_x_y_t[:,1:2]
t = train_x_y_t[:,2:3]
x.requires_grad_()
y.requires_grad_()
t.requires_grad_()

tensor([[0.3976],
        [0.7691],
        [0.9756],
        ...,
        [0.2023],
        [0.6423],
        [0.6873]], requires_grad=True)

## Get Data

In [4]:
def relative_l2(u_pred,u_real):
    l2 = np.linalg.norm(u_real-u_pred,2)/np.linalg.norm(u_real,2)
    return l2

In [5]:
import scipy.integrate
from numpy import exp
from math import sqrt
import math
def InIn(x_y_t):
    x1 = x_y_t[:,0:1]
    x2 = x_y_t[:,1:2]
    t = x_y_t[:,2:3]
    P = np.zeros_like(t)
    for i in range(len(t)):
        f = lambda y1,y2 : exp(-((x1[i]-y1)**2+(x2[i]-y2)**2)/(4*t[i]))*(y1-y2)
        p,err= scipy.integrate.dblquad(f, -np.inf, np.inf, lambda g : -np.inf, lambda h : np.inf)
        P[i] = p/(4*np.pi*t[i])
    return P

In [6]:
np.random.seed(5678)
test_x_y_t = lhs(3,500)
x1 = test_x_y_t[:,0:1]
x2 = test_x_y_t[:,1:2]
t_ = test_x_y_t[:,2:3]
test_u = np.zeros_like(t_)
for i in range(len(t_)):
    f = lambda y1,y2 : exp(-((x1[i]-y1)**2+(x2[i]-y2)**2)/(4*t_[i]))*(y1-y2)
    p,err= scipy.integrate.dblquad(f, -np.inf, np.inf, lambda g : -np.inf, lambda h : np.inf)
    test_u[i] = p/(4*np.pi*t_[i])


In [7]:
test_x_y_t = torch.from_numpy(test_x_y_t).float()

### Labeled Data

In [8]:
np.random.seed(234)
n = 8
gan_data_x_t = lhs(3,10)
#np.random.seed(1234)
np.random.seed(1234)
chosen_indices = np.random.choice(gan_data_x_t.shape[0], size=n, replace=False)
gan_data_x_t = gan_data_x_t[chosen_indices, :]
gan_data_u = InIn(gan_data_x_t)

In [9]:
gan_data_x_t = torch.from_numpy(gan_data_x_t).float()
gan_data_u = torch.from_numpy(gan_data_u).float()

# Method2
**$\text { GA - PINN }$**

<div class="alert alert-info">`loss_function：`

$\mathrm{L}_D=\frac{1}{J} \sum_{j=1}^J\left(1-D\left[\left(\mathbf{x}_T^{(j)}, u_T^{(j)}\right)\right]\right)+D\left[\left(x_L^{(j)}, G\left[x_L^{(j)}\right]\right)\right] \\
\mathrm{L}_G=\mathrm{L}_T+\frac{1}{J} \sum_{j=1}^J\left(1-D\left[\left(x_T^{(j)}, G\left[x_T^{(j)}\right]\right)\right]\right)\\
\mathrm{L}_{\text {PINN }}:=\mathrm{L}_f+\lambda_1 \mathrm{~L}_b \text { with } \mathrm{L}_b:=\sum_{i=1}^I \mathrm{~L}_{b_i}
$



</div>

In [10]:
torch.manual_seed(1234)
torch.cuda.manual_seed(1234)
np.random.seed(1234)

PINNs2 = NN_H2(3, 100, 4, 1)
PINNs2.apply(weights_init)
optimizer1 = optim.Adam([{'params': PINNs2.parameters()}], lr=1e-3)

discriminator= get_discriminator(4, 100, 1, 1)
discriminator.apply(weights_init)
optimizer2 = optim.Adam([{'params': discriminator.parameters(), 'weight_decay': 0.01}], lr=5e-3)


loss_all_2 = []
test_loss_2 = []

#########gpu############
discriminator.cuda()
gan_data_x_t = gan_data_x_t.cuda()
gan_data_u = gan_data_u.cuda()
bound_x_y_t = bound_x_y_t.cuda()
bound_x_y = bound_x_y.cuda()
PINNs2.cuda()
x = x.cuda()
y = y.cuda()
t = t.cuda()
#########gpu############

loss1_value = 1
it = 0
while  loss1_value>5e-6:
    PINNs2.cuda()
    
    ##############loss D############
    pre_H = PINNs2(gan_data_x_t)
    d_fake = discriminator(torch.cat((gan_data_x_t,pre_H.detach()),1))
    d_real = discriminator(torch.cat((gan_data_x_t,gan_data_u),1))
    
    loss_d = torch.mean(1-d_real)+torch.mean(d_fake)
    
    optimizer2.zero_grad()
    loss_d .backward()
    optimizer2.step()  

    ##############loss G ############
    pre_H = PINNs2(gan_data_x_t)
    d_fake = discriminator(torch.cat((gan_data_x_t,pre_H.detach()),1))
    loss_L = torch.mean(torch.square(pre_H - gan_data_u))+torch.mean(1-d_fake)
    
    optimizer1.zero_grad()
    loss_L.backward()
    optimizer1.step()  
    
    ##### loss_Bi  ######
    u_bound = PINNs2(bound_x_y_t)
    
    loss_bound = torch.mean(torch.square(u_bound-(bound_x_y[:,0:1]-bound_x_y[:,1:2])))
  
    ##### loss f  ######
    
    u_inside = PINNs2(torch.cat((x,y,t),1))
    u_x = autograd.grad(outputs=u_inside, inputs=x,
                              grad_outputs=torch.ones_like(u_inside),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]
    u_xx = autograd.grad(outputs=u_x, inputs=x,
                              grad_outputs=torch.ones_like(u_x),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]
    u_y = autograd.grad(outputs=u_inside, inputs=y,
                              grad_outputs=torch.ones_like(u_inside),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]
    u_yy = autograd.grad(outputs=u_y, inputs=y,
                              grad_outputs=torch.ones_like(u_y),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]  
    u_t = autograd.grad(outputs=u_inside, inputs=t,
                          grad_outputs=torch.ones_like(u_inside),
                          create_graph=True, retain_graph=True, only_inputs=True)[0]
    deata_u = u_xx+u_yy
    loss_f = torch.mean(torch.square(deata_u-u_t))
    
    loss = loss_bound+loss_f
    loss1_value = loss.item()

    loss_all_2.append(loss1_value)
    optimizer1.zero_grad()
    loss.backward()
    optimizer1.step()
    
    #########  test_loss NRMSE  #########
    PINNs2.cpu()
    test_loss =  relative_l2(PINNs2(test_x_y_t).detach().numpy(),test_u)
    test_loss_2.append(test_loss)
    
    if it % 100 == 0:
        print('It:', it, 'Loss:', loss.item())
    it = it + 1        
loss1_value    

It: 0 Loss: 0.18561820685863495
It: 100 Loss: 0.0006691083544865251
It: 200 Loss: 0.005303528625518084
It: 300 Loss: 0.0001416200102539733
It: 400 Loss: 6.369435868691653e-05
It: 500 Loss: 0.00015775191423017532
It: 600 Loss: 5.954594962531701e-05
It: 700 Loss: 0.0005049089668318629
It: 800 Loss: 6.258127541514114e-05
It: 900 Loss: 3.132211713818833e-05
It: 1000 Loss: 6.5719272242859e-05
It: 1100 Loss: 2.7881702408194542e-05
It: 1200 Loss: 5.86743226449471e-05
It: 1300 Loss: 2.2327498300001025e-05
It: 1400 Loss: 5.465965841722209e-06


4.991502919438062e-06

# Method3
$\text { PINN }^{\dagger}$

<div class="alert alert-info">objective function：

$\overline{\mathrm{L}}_{\text {PINN }} =\mathrm{L}_{\text {PINN }}+\lambda_2\mathrm{L}_T$



</div>

In [11]:
k = 2

In [12]:
torch.manual_seed(1234)
torch.cuda.manual_seed(1234)
np.random.seed(1234)

PINNs3 = NN_H2(3, 100, 4, 1)
PINNs3.apply(weights_init)
optimizer1 = optim.Adam([{'params': PINNs3.parameters()}], lr=1e-3)

loss_all_3 = []
test_loss_3 = []

#########gpu############
bound_x_y_t = bound_x_y_t.cuda()
bound_x_y = bound_x_y.cuda()
PINNs3.cuda()
x = x.cuda()
y = y.cuda()
t = t.cuda()
gan_data_x_t = gan_data_x_t.cuda()
gan_data_u = gan_data_u.cuda()
#########gpu############

loss1_value = 1
it = 0
while  loss1_value>5e-6:
    PINNs3.cuda()
    
    ##### loss_Bi  ######
    u_bound = PINNs3(bound_x_y_t)
    
    loss_bound = torch.mean(torch.square(u_bound-(bound_x_y[:,0:1]-bound_x_y[:,1:2])))
  
    ##### loss f  ######
    u_inside = PINNs3(torch.cat((x,y,t),1))
    u_x = autograd.grad(outputs=u_inside, inputs=x,
                              grad_outputs=torch.ones_like(u_inside),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]
    u_xx = autograd.grad(outputs=u_x, inputs=x,
                              grad_outputs=torch.ones_like(u_x),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]
    u_y = autograd.grad(outputs=u_inside, inputs=y,
                              grad_outputs=torch.ones_like(u_inside),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]
    u_yy = autograd.grad(outputs=u_y, inputs=y,
                              grad_outputs=torch.ones_like(u_y),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]  
    u_t = autograd.grad(outputs=u_inside, inputs=t,
                          grad_outputs=torch.ones_like(u_inside),
                          create_graph=True, retain_graph=True, only_inputs=True)[0]
    deata_u = u_xx+u_yy
    loss_f = torch.mean(torch.square(deata_u-u_t))

    #####loss PI#######
    loss = loss_bound+loss_f
    pre_H = PINNs3(gan_data_x_t)
    
    loss_p = k*torch.mean(torch.square(pre_H - gan_data_u))+loss
    
    
    loss1_value = loss.item()
    
    
    loss_all_3.append(loss1_value)
    optimizer1.zero_grad()
    loss_p.backward()
    optimizer1.step()
    
    #########  test_loss NRMSE  #########
    PINNs3.cpu()
    test_loss =  relative_l2(PINNs3(test_x_y_t).detach().numpy(),test_u)
    test_loss_3.append(test_loss)
    
    if it % 100 == 0:
        print('It:', it, 'Loss:', loss.item(), 'testloss:', test_loss)
    it = it + 1        
loss1_value    

It: 0 Loss: 0.32263725996017456 testloss: 0.824821223278985
It: 100 Loss: 0.0015198912005871534 testloss: 0.108716347013447
It: 200 Loss: 0.0005600953591056168 testloss: 0.08138574916635542
It: 300 Loss: 0.00047946328413672745 testloss: 0.059117248909874624
It: 400 Loss: 0.00013332019443623722 testloss: 0.05603419275386172
It: 500 Loss: 8.346936374437064e-05 testloss: 0.05546110957045734
It: 600 Loss: 6.345986912492663e-05 testloss: 0.05498074256902373
It: 700 Loss: 0.00012599346518982202 testloss: 0.04822985591464004
It: 800 Loss: 7.431671838276088e-05 testloss: 0.0461990752077216
It: 900 Loss: 5.486154623213224e-05 testloss: 0.04702357938555055
It: 1000 Loss: 4.422767597134225e-05 testloss: 0.046888109629137545
It: 1100 Loss: 0.0011177968699485064 testloss: 0.12792544696600974
It: 1200 Loss: 8.728187822271138e-05 testloss: 0.03552087160419719
It: 1300 Loss: 5.364811295294203e-05 testloss: 0.03787475944469299
It: 1400 Loss: 3.973874481744133e-05 testloss: 0.038737572324091404
It: 1500

4.985811756341718e-06

# Method4 
$\text { GA - PINN }^{\dagger}$

<div class="alert alert-info">objective function：

$
\mathrm{L}_D=\frac{1}{J} \sum_{j=1}^J\left(1-D\left[\left(\mathbf{x}_T^{(j)}, u_T^{(j)}\right)\right]\right)+D\left[\left(x_L^{(j)}, G\left[x_L^{(j)}\right]\right)\right] \\
\mathrm{L}_G=\mathrm{L}_T+\frac{1}{J} \sum_{j=1}^J\left(1-D\left[\left(x_T^{(j)}, G\left[x_T^{(j)}\right]\right)\right]\right)\\
\overline{\mathrm{L}}_{\text {PINN }} =\mathrm{L}_{\text {PINN }}+\lambda_2\mathrm{L}_T
$



</div>


In [13]:
torch.manual_seed(1234)
torch.cuda.manual_seed(1234)
np.random.seed(1234)

PINNs4 = NN_H2(3, 100, 4, 1)
PINNs4.apply(weights_init)
optimizer1 = optim.Adam([{'params': PINNs4.parameters()}], lr=1e-3)

discriminator= get_discriminator(4, 100, 1, 1)
discriminator.apply(weights_init)
optimizer2 = optim.Adam([{'params': discriminator.parameters(), 'weight_decay': 0.01}], lr=5e-3)

loss_all_4 = []
test_loss_4 = []

#########gpu############
discriminator.cuda()
gan_data_x_t = gan_data_x_t.cuda()
gan_data_u = gan_data_u.cuda()
bound_x_y_t = bound_x_y_t.cuda()
bound_x_y = bound_x_y.cuda()
PINNs4.cuda()
x = x.cuda()
y = y.cuda()
t = t.cuda()
#########gpu############


loss1_value = 1
it = 0
while  loss1_value>5e-6:
    PINNs4.cuda()
    
    ##############loss D############
    pre_H = PINNs4(gan_data_x_t)
    d_fake = discriminator(torch.cat((gan_data_x_t,pre_H.detach()),1))
    d_real = discriminator(torch.cat((gan_data_x_t,gan_data_u),1))
    
    loss_d = torch.mean(1-d_real)+torch.mean(d_fake)
    
    optimizer2.zero_grad()
    loss_d .backward()
    optimizer2.step()  
    
    ##############loss G############
    pre_H = PINNs4(gan_data_x_t)
    d_fake = discriminator(torch.cat((gan_data_x_t,pre_H.detach()),1))
    loss_L = torch.mean(torch.square(pre_H - gan_data_u))+torch.mean(1-d_fake)
    
    optimizer1.zero_grad()
    loss_L.backward()
    optimizer1.step()  

    ##### loss_Bi  ######
    u_bound = PINNs4(bound_x_y_t)
    
    loss_bound = torch.mean(torch.square(u_bound-(bound_x_y[:,0:1]-bound_x_y[:,1:2])))
  
    ##### loss f  ######
    
    u_inside = PINNs4(torch.cat((x,y,t),1))
    u_x = autograd.grad(outputs=u_inside, inputs=x,
                              grad_outputs=torch.ones_like(u_inside),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]
    u_xx = autograd.grad(outputs=u_x, inputs=x,
                              grad_outputs=torch.ones_like(u_x),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]
    u_y = autograd.grad(outputs=u_inside, inputs=y,
                              grad_outputs=torch.ones_like(u_inside),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]
    u_yy = autograd.grad(outputs=u_y, inputs=y,
                              grad_outputs=torch.ones_like(u_y),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]  
    u_t = autograd.grad(outputs=u_inside, inputs=t,
                          grad_outputs=torch.ones_like(u_inside),
                          create_graph=True, retain_graph=True, only_inputs=True)[0]
    deata_u = u_xx+u_yy
    loss_f = torch.mean(torch.square(deata_u-u_t))
    
    #####loss PI#######
    loss = loss_bound+loss_f
    pre_H = PINNs4(gan_data_x_t)
    
    loss_p = k*torch.mean(torch.square(pre_H - gan_data_u))+loss
    
    
    loss1_value = loss.item()
    
    
    loss_all_4.append(loss1_value)
    optimizer1.zero_grad()
    loss_p.backward()
    optimizer1.step()
    
    #########  test_loss NRMSE  #########
    PINNs4.cpu()
    test_loss =  relative_l2(PINNs4(test_x_y_t).detach().numpy(),test_u)
    test_loss_4.append(test_loss)
    
    if it % 100 == 0:
        print('It:', it, 'Loss:', loss.item(), 'testloss:', test_loss)
    it = it + 1        
loss1_value    

It: 0 Loss: 0.18561820685863495 testloss: 0.8205320214833214
It: 100 Loss: 0.0010124279651790857 testloss: 0.050406782751726796
It: 200 Loss: 0.0004476501781027764 testloss: 0.03748330193688663
It: 300 Loss: 0.0006130391266196966 testloss: 0.026121920890509315
It: 400 Loss: 0.00021685942192561924 testloss: 0.02149178547940145
It: 500 Loss: 0.00011598061973927543 testloss: 0.021397416229167188
It: 600 Loss: 0.0002868190931621939 testloss: 0.01994457439845682
It: 700 Loss: 0.000123118152259849 testloss: 0.017440117246922564
It: 800 Loss: 7.030732376733795e-05 testloss: 0.017582015342534493
It: 900 Loss: 0.00014735228614881635 testloss: 0.01487726049476496
It: 1000 Loss: 6.908128852955997e-05 testloss: 0.014796501979881275
It: 1100 Loss: 9.932083776220679e-05 testloss: 0.013977067597292638
It: 1200 Loss: 4.3130428821314126e-05 testloss: 0.014010427537697568
It: 1300 Loss: 5.685388168785721e-05 testloss: 0.012424479975605725
It: 1400 Loss: 6.80868743074825e-06 testloss: 0.01123835520523172

4.978110155207105e-06

In [14]:
test_loss_2 = np.array(test_loss_2)
test_loss_3 = np.array(test_loss_3)
test_loss_4 = np.array(test_loss_4)

In [16]:
np.save('../../experimental_data/J_8/test_loss_2',test_loss_2)
np.save('../../experimental_data/J_8/test_loss_3',test_loss_3)
np.save('../../experimental_data/J_8/test_loss_4',test_loss_4)